In [22]:
import torch
from copy import deepcopy

In [3]:
#Create the positive definite matrix.
N = 5
L = torch.randn(N, N)
A = L @ L.t()

In [25]:
import torch
import torch.optim as optim

# Define the quadratic form function
def quadratic_form(x, A):
    return 0.5 * torch.matmul(torch.matmul(x.t(), A), x)

# Define the gradient descent function, returning optimal point AND number of iterations
def gradient_descent(A, learning_rate, x0=None, max_iter=1000):
    # Initialize the starting point
    x = x0 if x0 is not None else torch.randn(A.shape[0], 1)
    x.requires_grad = True
        
    # Perform gradient descent
    precision = 1e-6
    prev_loss = float('inf')
    iterations = 0
    while iterations < max_iter:
        # Compute the loss
        loss = quadratic_form(x, A)
        
        # Check for convergence
        if torch.abs(loss - prev_loss) < precision:
            break
        
        # Update the parameters
        x.grad = None
        loss.backward()
        with torch.no_grad():
            x -= learning_rate * x.grad
        iterations += 1
        
        if iterations%10000 == 0:
            print(loss.item(), x)

        
        # Update the previous loss
        prev_loss = loss
    
    # Return the optimized solution
    return x, iterations

# Define a symmetric matrix with fixed eigenvalues
A = torch.diag(torch.tensor([5., 4., 3., 2., 1.]))
print(A)

# Theoretically optimal learning rate
lr = 2.0 / (torch.max(A) + torch.min(A))
print(lr)

# Perform gradient descent
x_star, iterations = gradient_descent(A, lr, x0 = torch.tensor([1., 1, 1, 1, 1]))
print(f"{x_star=}, {iterations=}")



tensor([[5., 0., 0., 0., 0.],
        [0., 4., 0., 0., 0.],
        [0., 0., 3., 0., 0.],
        [0., 0., 0., 2., 0.],
        [0., 0., 0., 0., 1.]])
tensor(0.4000)
x_star=tensor([-1.0000e+00, -4.7019e-04, -3.2768e-11,  3.2768e-11,  4.7018e-04],
       requires_grad=True), iterations=15


In [28]:
#Find best learning rate experimentally
x0 = torch.tensor([1., 1, 1, 1, 1])
learning_rates = torch.linspace(0.01, 1, 100)
best_lr = None
best_it = 9999999999
for learning_rate in learning_rates:
    x_star, iterations = gradient_descent(A, learning_rate, x0=deepcopy(x0))
    converged = (iterations < 1000)
    print(f"{learning_rate=}, {iterations=}, {converged=}")

    #Update best learning rate
    if iterations <= best_it:
        best_lr = learning_rate
        best_it = iterations
print(f"Fastest convergence using lr={best_lr} in {best_it} iterations")

learning_rate=tensor(0.0100), iterations=459, converged=True
learning_rate=tensor(0.0200), iterations=246, converged=True
learning_rate=tensor(0.0300), iterations=170, converged=True
learning_rate=tensor(0.0400), iterations=131, converged=True
learning_rate=tensor(0.0500), iterations=107, converged=True
learning_rate=tensor(0.0600), iterations=90, converged=True
learning_rate=tensor(0.0700), iterations=78, converged=True
learning_rate=tensor(0.0800), iterations=69, converged=True
learning_rate=tensor(0.0900), iterations=62, converged=True
learning_rate=tensor(0.1000), iterations=56, converged=True
learning_rate=tensor(0.1100), iterations=51, converged=True
learning_rate=tensor(0.1200), iterations=47, converged=True
learning_rate=tensor(0.1300), iterations=44, converged=True
learning_rate=tensor(0.1400), iterations=41, converged=True
learning_rate=tensor(0.1500), iterations=38, converged=True
learning_rate=tensor(0.1600), iterations=36, converged=True
learning_rate=tensor(0.1700), itera

In [49]:
def adaptive_lr_gd():
    # Do gradient descent with adaptive learning rate.
    x = deepcopy(x0)
    x.requires_grad = True
    precision = 1e-6
    lr = torch.tensor([0.01], requires_grad=True)
    lr_lr = 0.001
    iterations = 0
    prev_loss = 999999.99

    while iterations < 100:
        # Compute the loss
        loss = quadratic_form(x, A)
        
        # Check for convergence
        if torch.abs(loss - prev_loss) < precision:
            break
        
        # Update the parameters
        x.grad = None
        x.retain_grad()
        lr.grad = None
        loss.backward()
        if iterations%10 == 0:
            print(f"{loss=}, {x=}, {x.grad=}, {lr.item()=}")

        #Update learning rate
        if iterations > 0:
            with torch.no_grad():
                lr -= lr_lr * lr.grad

        #Update x
        x = x.detach() - lr * x.grad.detach()

        iterations += 1
        
        # Update the previous loss
        prev_loss = loss
        
    print(x, iterations, lr)

adaptive_lr_gd()

loss=tensor(7.5000, grad_fn=<MulBackward0>), x=tensor([1., 1., 1., 1., 1.], requires_grad=True), x.grad=tensor([5., 4., 3., 2., 1.]), lr.item()=0.009999999776482582
loss=tensor(0.0612, grad_fn=<MulBackward0>), x=tensor([2.9880e-04, 3.1261e-03, 1.9641e-02, 8.9311e-02, 3.2455e-01],
       grad_fn=<SubBackward0>), x.grad=tensor([0.0015, 0.0125, 0.0589, 0.1786, 0.3246]), lr.item()=0.1293065845966339
loss=tensor(0.0033, grad_fn=<MulBackward0>), x=tensor([8.5387e-09, 2.0719e-06, 1.4182e-04, 4.4272e-03, 8.0856e-02],
       grad_fn=<SubBackward0>), x.grad=tensor([4.2694e-08, 8.2876e-06, 4.2547e-04, 8.8544e-03, 8.0856e-02]), lr.item()=0.12987931072711945
loss=tensor(0.0002, grad_fn=<MulBackward0>), x=tensor([2.3888e-13, 1.3563e-09, 1.0166e-06, 2.1858e-04, 2.0109e-02],
       grad_fn=<SubBackward0>), x.grad=tensor([1.1944e-12, 5.4251e-09, 3.0498e-06, 4.3715e-04, 2.0109e-02]), lr.item()=0.12990854680538177
loss=tensor(1.2504e-05, grad_fn=<MulBackward0>), x=tensor([6.6750e-18, 8.8720e-13, 7.2841e-